# Teoría de funcionales de la densidad (DFT)

La teoría de los funcionales de la densidad se basa en que la densidad electrónica contiene toda la información del sistema, por lo que se puede extraer de ella la energía mediante un funcional

$$
E[\rho] = T_s[\rho] + U[\rho] + V_{nuc}[\rho] + E_{xc}[\rho]
$$

Donde
- $E[\rho]$ es la energía del sistema 
- $T_s[\rho]$ es la energía cinética de Kohn-Sham
- $U[\rho]$ es la energía de Hartree
- $V[\rho]$ es la interacción núcleo electrón
- $E_{xc} [\rho]$ es la energía de intercambio correlación

En la formulación de Kohn y Sham, la densidad electrónica ($\rho(r)$) se calcula a partir de los orbitales de Kohn-Sham ($\psi^{KS}_i(r)$)

$$
\rho(r) = \sum_i^{N} |\psi^{KS}_i(r)|^2
$$


```{warning}
Los teoremas de Hohenberg y Kohn, y Kohn y Sham prueban que existe un funcional universal que conecta a la densidad electrónica con la energía del sistema, pero este funcional no se conoce, particularmente por la parte de $E_{xc}[\rho]$. Se han realizado diversas propuestas de como construir este funcional, por lo que en la práctica existen cientos de funcionales de DFT.
```

```{note}
No existe el mejor funcional, sino que depende del sistema químico que se esté estudiando.
```

En general los funcionales aproximan de diferente manera el intercambio y la correlación, dependiendo de como se aproxime el funcional, estos se pueden clasificar en diferentes categorías. La clasificación fue propuesta por Perdew y se conoce como la `escalera de Jacob`.

```{warning}
Evaluar la contribución del funcional de intercambio-correlación requiere de realizar integración numérica. Esto se hace con un mallado entorno a la molécula. Existen diversos esquemas para colocar los puntos en el mallado, y para seleccionar cuantos puntos poner, estos puede cambiar de un software a otro, e incluso entre diferentes versiones de un mismo software.
```

**Importe psi4**

In [1]:
#Importe psi4

In [2]:
import psi4

Para ejemplificar el uso de estos funcionales, declare la molécula de agua.

```
h2o = psi4.geometry("""
0 1
O    0.0000    0.0000    0.1173
H    0.0000    0.7572   -0.4692
H    0.0000   -0.7572   -0.4692 
units angstrom
""")
```

In [3]:
# h2o

In [4]:
h2o = psi4.geometry("""
0 1
O    0.0000    0.0000    0.1173
H    0.0000    0.7572   -0.4692
H    0.0000   -0.7572   -0.4692 
units angstrom
""")

## Aproximación Local de la Densidad (LDA)

Fue una de las primeras aproximaciones y ya ha sido superada. En este caso, el intercambio se calcula mediante

$$
E_x^{LDA} = -\frac{3}{4} \left( \frac{3}{\pi} \right)^{1/3} \int \rho^{4/3} (r) dr 
$$

y la correlación se calcula mediante el funcional de Vosko, Wilk y Nusair
$$
E_c^{LDA} = \int \varepsilon_c^{VWN} dr
$$

Haga un cálculo de energía con SVWN y la base 6-311G con la siguiente instrucción
```
psi4.energy('SVWN/6-311G')
```

In [5]:
# SVWN

In [6]:
psi4.energy('SVWN/6-311G')


Scratch directory: /tmp/

*** tstart() called on B450-AORUS-PRO-WIFI
*** at Thu Jun  2 11:38:09 2022

   => Loading Basis Set <=

    Name: 6-311G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line   126 file /home/jfhlewyee/anaconda3/envs/jb/share/psi4/basis/6-311g.gbs 
    atoms 2-3 entry H          line    21 file /home/jfhlewyee/anaconda3/envs/jb/share/psi4/basis/6-311g.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RKS Reference
                        1 Threads,    500 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular point group: c2v
    Full point group: C2v

    Geometry (in Angstrom), charge = 0, multiplicity = 1:

       Center              X                  Y                   Z          

   @DF-RKS iter   1:   -75.83750725761067   -5.07058e-02   6.54936e-02 DIIS/ADIIS
   @DF-RKS iter   2:   -75.31953042974517    5.17977e-01   1.12927e-01 ADIIS
   @DF-RKS iter   3:   -76.00860317866201   -6.89073e-01   2.84616e-02 DIIS/ADIIS
   @DF-RKS iter   4:   -76.04484938328758   -3.62462e-02   4.35363e-03 DIIS/ADIIS
   @DF-RKS iter   5:   -76.04570472895017   -8.55346e-04   4.21069e-04 DIIS/ADIIS
   @DF-RKS iter   6:   -76.04571320376117   -8.47481e-06   1.63808e-05 DIIS
   @DF-RKS iter   7:   -76.04571321527150   -1.15103e-08   9.88756e-07 DIIS
  Energy and wave function converged.


  ==> Post-Iterations <==

   Electrons on quadrature grid:
      Ntotal   =    9.9999999211 ; deviation = -7.889e-08 

    Orbital Energies [Eh]
    ---------------------

    Doubly Occupied:                                                      

       1A1   -18.592943     2A1    -0.948080     1B2    -0.502246  
       3A1    -0.322093     1B1    -0.259374  

    Virtual:                          

-76.0457132152715


*** tstop() called on B450-AORUS-PRO-WIFI at Thu Jun  2 11:38:09 2022
Module time:
	user time   =       0.34 seconds =       0.01 minutes
	system time =       0.04 seconds =       0.00 minutes
	total time  =          0 seconds =       0.00 minutes
Total time:
	user time   =       0.34 seconds =       0.01 minutes
	system time =       0.04 seconds =       0.00 minutes
	total time  =          0 seconds =       0.00 minutes


## Aproximación de Gradientes Generalizados (GGA)

Esto calcula la energía con base en la densidad electrónica y su gradiente

$$
E_{xc}^{GGA} = -\int \varepsilon_{xc}^{GGA} (\rho,\nabla \rho) dr
$$

Para ello separa los funcionales en intercambio y correlación.

$$
\varepsilon_{xc}^{GGA} = \varepsilon_{x}^{GGA} + \varepsilon_{c}^{GGA}
$$

Algunos funcionales de intercambio GGA son
- PWx86: Perdew-Wang 1986
- B88: Becke 1988
- PWx91: Perdew-Wang 1991
- PBE: Perdew-Burke-Ernzerhof
    
Algunos funcionales de correlación GGA son
- LYP: Lee-Yang-Parr
- Pc86: Perdew 1986
- PWc91: Perdew-Wang 1991
- PBE: Perdew-Burke-Ernzerhof
    
La combinación de estos funcionales genera los funcionales GGA. **Haga un cálculo de energía con PBE y la base 6-311G con la siguiente instrucción**
```
psi4.energy('PBE/6-311G')
```

In [7]:
psi4.energy('pbe/6-311G')


Scratch directory: /tmp/

*** tstart() called on B450-AORUS-PRO-WIFI
*** at Thu Jun  2 11:38:09 2022

   => Loading Basis Set <=

    Name: 6-311G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line   126 file /home/jfhlewyee/anaconda3/envs/jb/share/psi4/basis/6-311g.gbs 
    atoms 2-3 entry H          line    21 file /home/jfhlewyee/anaconda3/envs/jb/share/psi4/basis/6-311g.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RKS Reference
                        1 Threads,    500 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular point group: c2v
    Full point group: C2v

    Geometry (in Angstrom), charge = 0, multiplicity = 1:

       Center              X                  Y                   Z          

   => Loading Basis Set <=

    Name: (6-311G AUX)
    Role: JKFIT
    Keyword: DF_BASIS_SCF
    atoms 1   entry O          line   229 file /home/jfhlewyee/anaconda3/envs/jb/share/psi4/basis/cc-pvtz-jkfit.gbs 
    atoms 2-3 entry H          line    51 file /home/jfhlewyee/anaconda3/envs/jb/share/psi4/basis/cc-pvtz-jkfit.gbs 

  ==> Integral Setup <==

  DFHelper Memory: AOs need 0.001 GiB; user supplied 0.331 GiB. Using in-core AOs.

  ==> MemDFJK: Density-Fitted J/K Matrices <==

    J tasked:                   Yes
    K tasked:                    No
    wK tasked:                   No
    OpenMP threads:               1
    Memory [MiB]:               338
    Algorithm:                 Core
    Schwarz Cutoff:           1E-12
    Mask sparsity (%):       0.0000
    Fitting Condition:        1E-10

   => Auxiliary Basis Set <=

  Basis Set: (6-311G AUX)
    Blend: CC-PVTZ-JKFIT
    Number of shells: 45
    Number of basis functions: 139
    Number of Cartesian functions: 166
    Spher

  Minimum eigenvalue in the overlap matrix is 3.8481855116E-02.
  Reciprocal condition number of the overlap matrix is 1.0514592286E-02.
    Using symmetric orthogonalization.

  ==> Pre-Iterations <==

  SCF Guess: Superposition of Atomic Densities via on-the-fly atomic UHF (no occupation information).

   -------------------------
    Irrep   Nso     Nmo    
   -------------------------
     A1        10      10 
     A2         0       0 
     B1         3       3 
     B2         6       6 
   -------------------------
    Total      19      19
   -------------------------

  ==> Iterations <==

                           Total Energy        Delta E     RMS |[F,P]|

   @DF-RKS iter SAD:   -76.05808687398583   -7.60581e+01   0.00000e+00 
   @DF-RKS iter   1:   -76.12655589141131   -6.84690e-02   6.41352e-02 DIIS/ADIIS
   @DF-RKS iter   2:   -75.62770920108809    4.98847e-01   1.09611e-01 ADIIS
   @DF-RKS iter   3:   -76.29253535917977   -6.64826e-01   2.74487e-02 DIIS/ADIIS
   @DF-R

   @DF-RKS iter   6:   -76.32774682582524   -7.03213e-06   1.64132e-05 DIIS
   @DF-RKS iter   7:   -76.32774683796953   -1.21443e-08   6.72488e-07 DIIS
  Energy and wave function converged.


  ==> Post-Iterations <==

   Electrons on quadrature grid:
      Ntotal   =    9.9999999104 ; deviation = -8.958e-08 

    Orbital Energies [Eh]
    ---------------------

    Doubly Occupied:                                                      

       1A1   -18.723192     2A1    -0.934052     1B2    -0.480902  
       3A1    -0.302844     1B1    -0.236425  

    Virtual:                                                              

       4A1     0.003413     2B2     0.084152     3B2     0.344734  
       5A1     0.378494     2B1     0.719640     6A1     0.740365  
       4B2     0.847962     7A1     1.109018     8A1     2.068045  
       5B2     2.114866     3B1     4.637170     9A1     4.742631  
       6B2     4.857520    10A1    49.345078  

    Final Occupation by Irrep:
             A1 

-76.32774683796953


*** tstop() called on B450-AORUS-PRO-WIFI at Thu Jun  2 11:38:09 2022
Module time:
	user time   =       0.47 seconds =       0.01 minutes
	system time =       0.01 seconds =       0.00 minutes
	total time  =          0 seconds =       0.00 minutes
Total time:
	user time   =       0.83 seconds =       0.01 minutes
	system time =       0.05 seconds =       0.00 minutes
	total time  =          0 seconds =       0.00 minutes


## Aproximación meta-GGA

Los funcionales meta-GGA usan la densidad electrónica, el gradiente de la densidad electrónica, y el laplaciano de la densidad electrónica.

$$
E_{xc}^{meta-GGA} = -\int \varepsilon_{xc}^{meta-GGA} (\rho,\nabla \rho,\nabla^2 \rho) dr
$$

Algunos ejemplos son:
- B95: Becke 1995
- TPSS: Tau-Perdew-Staroverov-Scuseria

**Haga un cálculo de energía con TPSS y la base 6-311G con la siguiente instrucción**
```
psi4.energy('TPSS/6-311G')
```

In [8]:
# TPSS

In [9]:
psi4.energy('TPSS/6-311G')


Scratch directory: /tmp/

*** tstart() called on B450-AORUS-PRO-WIFI
*** at Thu Jun  2 11:38:09 2022

   => Loading Basis Set <=

    Name: 6-311G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line   126 file /home/jfhlewyee/anaconda3/envs/jb/share/psi4/basis/6-311g.gbs 
    atoms 2-3 entry H          line    21 file /home/jfhlewyee/anaconda3/envs/jb/share/psi4/basis/6-311g.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RKS Reference
                        1 Threads,    500 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular point group: c2v
    Full point group: C2v

    Geometry (in Angstrom), charge = 0, multiplicity = 1:

       Center              X                  Y                   Z          

   @DF-RKS iter SAD:   -76.13236875156562   -7.61324e+01   0.00000e+00 
   @DF-RKS iter   1:   -76.22349882312619   -9.11301e-02   6.28939e-02 DIIS/ADIIS
   @DF-RKS iter   2:   -75.81858472735249    4.04914e-01   1.02073e-01 ADIIS
   @DF-RKS iter   3:   -76.39132781978782   -5.72743e-01   2.35718e-02 DIIS/ADIIS
   @DF-RKS iter   4:   -76.41654416581284   -2.52163e-02   3.85010e-03 DIIS/ADIIS


   @DF-RKS iter   5:   -76.41723114646567   -6.86981e-04   2.85777e-04 DIIS/ADIIS
   @DF-RKS iter   6:   -76.41723503389798   -3.88743e-06   5.01882e-06 DIIS
   @DF-RKS iter   7:   -76.41723503492187   -1.02389e-09   1.27723e-07 DIIS
  Energy and wave function converged.


  ==> Post-Iterations <==

   Electrons on quadrature grid:
      Ntotal   =    9.9999999095 ; deviation = -9.045e-08 

    Orbital Energies [Eh]
    ---------------------

    Doubly Occupied:                                                      

       1A1   -18.863827     2A1    -0.956505     1B2    -0.487222  
       3A1    -0.312034     1B1    -0.244459  

    Virtual:                                                              

       4A1     0.011915     2B2     0.091732     3B2     0.352344  
       5A1     0.386412     2B1     0.735694     6A1     0.758241  
       4B2     0.869035     7A1     1.127509     8A1     2.144273  
       5B2     2.190459     3B1     4.708413     9A1     4.817659  
       6B2   

-76.41723503492187


*** tstop() called on B450-AORUS-PRO-WIFI at Thu Jun  2 11:38:11 2022
Module time:
	user time   =       1.18 seconds =       0.02 minutes
	system time =       0.02 seconds =       0.00 minutes
	total time  =          2 seconds =       0.03 minutes
Total time:
	user time   =       2.03 seconds =       0.03 minutes
	system time =       0.08 seconds =       0.00 minutes
	total time  =          2 seconds =       0.03 minutes


## Funcionales Híbridos

Mezclan un funcional de intercambio con el `intercambio de Hartree-Fock` en alguna proporción.

Algunos ejemplos de estos funcionales son:

- B3LYP
- PBE0
- M05-2X y M06-2X
- TPSSh

**Haga un cálculo de energía con M06-2X y la base 6-311G con la siguiente instrucción**
```
psi4.energy('M062X/6-311G')
```

In [10]:
#M062X

In [11]:
psi4.energy('M062X/6-311G')


Scratch directory: /tmp/

*** tstart() called on B450-AORUS-PRO-WIFI
*** at Thu Jun  2 11:38:11 2022

   => Loading Basis Set <=

    Name: 6-311G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line   126 file /home/jfhlewyee/anaconda3/envs/jb/share/psi4/basis/6-311g.gbs 
    atoms 2-3 entry H          line    21 file /home/jfhlewyee/anaconda3/envs/jb/share/psi4/basis/6-311g.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RKS Reference
                        1 Threads,    500 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular point group: c2v
    Full point group: C2v

    Geometry (in Angstrom), charge = 0, multiplicity = 1:

       Center              X                  Y                   Z          

  DFHelper Memory: AOs need 0.001 GiB; user supplied 0.278 GiB. Using in-core AOs.

  ==> MemDFJK: Density-Fitted J/K Matrices <==

    J tasked:                   Yes
    K tasked:                   Yes
    wK tasked:                   No
    OpenMP threads:               1
    Memory [MiB]:               284
    Algorithm:                 Core
    Schwarz Cutoff:           1E-12
    Mask sparsity (%):       0.0000
    Fitting Condition:        1E-10

   => Auxiliary Basis Set <=

  Basis Set: (6-311G AUX)
    Blend: CC-PVTZ-JKFIT
    Number of shells: 45
    Number of basis functions: 139
    Number of Cartesian functions: 166
    Spherical Harmonics?: true
    Max angular momentum: 4

  Cached 100.0% of DFT collocation blocks in 0.088 [GiB].

  Minimum eigenvalue in the overlap matrix is 3.8481855116E-02.
  Reciprocal condition number of the overlap matrix is 1.0514592286E-02.
    Using symmetric orthogonalization.

  ==> Pre-Iterations <==

  SCF Guess: Superposition of Atomic Dens

   @DF-RKS iter   4:   -76.37840581773118   -6.24785e-05   1.78179e-04 DIIS/ADIIS


   @DF-RKS iter   5:   -76.37840818416221   -2.36643e-06   1.34487e-05 DIIS
   @DF-RKS iter   6:   -76.37840821397789   -2.98157e-08   1.95764e-06 DIIS
   @DF-RKS iter   7:   -76.37840821474991   -7.72019e-10   2.27077e-07 DIIS
  Energy and wave function converged.


  ==> Post-Iterations <==

   Electrons on quadrature grid:
      Ntotal   =    9.9999999195 ; deviation = -8.052e-08 

    Orbital Energies [Eh]
    ---------------------

    Doubly Occupied:                                                      

       1A1   -19.606960     2A1    -1.142308     1B2    -0.608173  
       3A1    -0.446058     1B1    -0.385389  

    Virtual:                                                              

       4A1     0.058116     2B2     0.133409     3B2     0.424957  
       5A1     0.459890     2B1     0.832824     6A1     0.853903  
       4B2     0.959320     7A1     1.225442     8A1     2.173187  
       5B2     2.212067     3B1     4.881167     9A1     4.985598  
       6B2     5.09

   A2    B1    B2 
    DOCC [     3,    0,    1,    1 ]

  @DF-RKS Final Energy:   -76.37840821474991

   => Energetics <=

    Nuclear Repulsion Energy =              9.1895337585934787
    One-Electron Energy =                -123.0465209685796140
    Two-Electron Energy =                  41.9684561839964516
    DFT Exchange-Correlation Energy =      -4.4898771887602376
    Empirical Dispersion Energy =           0.0000000000000000
    VV10 Nonlocal Energy =                  0.0000000000000000
    Total Energy =                        -76.3784082147499106

Computation Completed


Properties will be evaluated at   0.000000,   0.000000,   0.000000 [a0]

Properties computed using the SCF density matrix


 Multipole Moments:

 ------------------------------------------------------------------------------------
     Multipole            Electronic (a.u.)      Nuclear  (a.u.)        Total (a.u.)
 ------------------------------------------------------------------------------------

 L = 1.

-76.37840821474991


*** tstop() called on B450-AORUS-PRO-WIFI at Thu Jun  2 11:38:12 2022
Module time:
	user time   =       0.92 seconds =       0.02 minutes
	system time =       0.01 seconds =       0.00 minutes
	total time  =          1 seconds =       0.02 minutes
Total time:
	user time   =       2.97 seconds =       0.05 minutes
	system time =       0.09 seconds =       0.00 minutes
	total time  =          3 seconds =       0.05 minutes


## Referencias

- P. Hohenberg and W. Kohn, Phys. Rev. 136, B864 (1964).
- P. Atkins, Molecular Quantum Mechanics (Oxford University Press, 2005).
- D. Rappoport, N. R. M. Crawford, F. Furche, y K. Burke, Which functional should I choose?, (2008).
- W. Koch y M.C. Holthausen, A Chemist’s Guide to Density Functional Theory, (2001).
- K. Burke y L.O. Wagner, Int. J. Quantum Chem. 113, 96 (2013).
- K. Burke, J. Chem. Phys. 136, (2012).